In [3]:
import matplotlib.pyplot as plt
import numpy as np
import random as rand
plt.style.use('seaborn')

In [4]:
def dudc(P,Q):
    P.sort()
    Q.sort()
    I = []
    nb_p = 0
    if len(Q) == 0 : 
        print("Q is empty set")
        return I
    elif len(Q)==1 : 
        for p in P : 
            if Q[0] <= p[0] and p[0] <= Q[1] : nb_p += 1
        if nb_p == len(P) : return Q
    else : 
        for p in P :
            for i in range(len(Q)):
                if i == len(Q)-1 :
                    if Q[i][0] <= p[0] and p[0] <= Q[i][1] : 
                        nb_p += 1
                        I.append(Q[len(Q)-1])
                elif p[0] <= Q[i][1] and p[0] < Q[i+1][0] : 
                    nb_p += 1 
                    I.append(Q[i])
                    break
            
        if nb_p == len(P) : return I
        else :
            print("We can't cover all the points")
            return I

In [5]:
P = [(1.5,0), (2.5,0), (1.8,0), (0.3,0), (2.54,0)]
Q = [(1.2,1.6),(1.6,2),(1.8,2.2),(2.5,2.9),(0,0.4),(1.4,1.8),(0.01,0.41)]
print("Interval = ", dudc(P,Q))

Interval =  [(0.01, 0.41), (1.4, 1.8), (1.8, 2.2), (2.5, 2.9), (2.5, 2.9)]


In [6]:
class Circle:
    
    def __init__(self, center,  radius=1):
        self.center = center
        self.radius = radius
        self.covered = None

    def cover_point(self, point):
        return (((self.center[0] - point[0]) ** 2 + (self.center[1] - point[1]) ** 2) ** 0.5 <= self.radius)

In [7]:
class Node(object):
    
    def __init__(self, parent, value, used_circles, name=(0,0)):
        self.parent = parent
        self.name = name
        self.value = value
        self.used_circles = used_circles
        self.child1 = None # take
        self.child2 = None # don't take
    
    def add_child(self, circle):
        self.child1 = Node(self, self.value, self.used_circles, (self.name[0]+1,1))
        used_circles = list(self.used_circles)
        used_circles.remove(circle)
        self.child2 = Node(self, self.value-1, used_circles, (self.name[0]+1,2))
    
    def coveredPoints(self, P):
        for i in self.used_circles :
            nb = 0
            for p in P:
                for circle in self.used_circles:
                    if circle.cover_point(p):
                        nb += 1
                        break
        return nb
            
class Tree:
    
    def __init__(self, root, P, Q):
        self.root = root
        self.P = P
        self.Q = Q
        self.best_sol = Q
    
    def isRealisable(self):
        if len(P) != self.root.coveredPoints(self.P): return False
        return True
    
    def isSatisfied(self, node):
        return ((node.coveredPoints(self.P) == len(P)) and node.value <= len(self.best_sol))
        

def createTree(P,Q):
    def count(disk, P):
        return sum([int(disk.cover_point(p)) for p in P])
    
    dictDisk = dict()
    for disk in Q :dictDisk[disk] = count(disk,P)
    circles = [item[0] for item in sorted(dictDisk.items() ,  key=lambda x: x[1], reverse=True)]
    i = 0
    tree = Tree(Node(None, len(Q), circles),P,Q)
    currentNode = tree.root
    if tree.isRealisable():
        for circle in circles : 
            currentNode.add_child(circle)
            if tree.isSatisfied(currentNode.child2):
                tree.best_sol = currentNode.child2.used_circles
                toVisit = currentNode.child2
            if tree.isSatisfied(currentNode.child1) :
                tree.best_sol = currentNode.child1.used_circles
                toVisit = currentNode.child1
            currentNode = toVisit
    return tree

In [8]:
P = [(1.5, 2.5), (3.5, 3.5), (4.5, 2.5), (4.5, 3.5), (5.5, 3.5), (5.5, 3.4), (1.5, 2.6), (2, 1)]
Q = [Circle((0, 0)), Circle((2, 2)), Circle((4.5, 2)), Circle((5, 3)), Circle((6, 4)), Circle((4,3))]

In [9]:
tree = createTree(P,Q)
toVisit = [tree.root]
while toVisit != []:
    current = toVisit[0]
    print(current.name)
    for circle in current.used_circles: print(circle.center, end = " ")
    print()
    if current.child1 != None :
        toVisit.append(current.child1)
    if current.child2 != None :
        toVisit.append(current.child2)
    toVisit.pop(0)

(0, 0)
(5, 3) (2, 2) (4, 3) (6, 4) (4.5, 2) (0, 0) 
(1, 1)
(5, 3) (2, 2) (4, 3) (6, 4) (4.5, 2) (0, 0) 
(1, 2)
(2, 2) (4, 3) (6, 4) (4.5, 2) (0, 0) 
(2, 1)
(2, 2) (4, 3) (6, 4) (4.5, 2) (0, 0) 
(2, 2)
(4, 3) (6, 4) (4.5, 2) (0, 0) 
(3, 1)
(2, 2) (4, 3) (6, 4) (4.5, 2) (0, 0) 
(3, 2)
(2, 2) (6, 4) (4.5, 2) (0, 0) 
(4, 1)
(2, 2) (4, 3) (6, 4) (4.5, 2) (0, 0) 
(4, 2)
(2, 2) (4, 3) (4.5, 2) (0, 0) 
(5, 1)
(2, 2) (4, 3) (6, 4) (4.5, 2) (0, 0) 
(5, 2)
(2, 2) (4, 3) (6, 4) (0, 0) 
(6, 1)
(2, 2) (4, 3) (6, 4) (0, 0) 
(6, 2)
(2, 2) (4, 3) (6, 4) 
